## Introdução
Conforme detalhei na Seção 4.2.3 da minha [monografia](https://github.com/anajbellini/SentimentAnalysis-MovieReviews/blob/master/documents/tcc-monografia-ajb.pdf), na etapa de processamento de texto do meu trabalho, defini 4 _pipelines_ distintos, sendo eles:

1. Aplicação de _stemming_, representando textos por _Bag-of-Words_;
2. Aplicação de _stemming_, representando textos por TF-IDF;
3. Aplicação de _lematização_, representando textos por _Bag-of-Words_; e
4. Aplicação de _lematização_, representando textos por TF-IDF.

A ordem de aplicação dos métodos pode ser vista abaixo, para melhor compreensão:

<center><img src="https://i.ibb.co/JxkJ377/fluxo.png" width="350"></center>

Neste Notebook, vou demonstrar os códigos que utilizei para cada método, para caso alguém esteja aprendendo Python e/ou processamento de linguagem natural e queira saber como fiz.

---

## Etapas Básicas
Não importa qual _pipeline_ estejamos usando, todos eles têm 3 etapas em comum: conversão para caixa baixa, remoção de pontuação e de _stop words_.

Para demonstrar o funcionamento destas três, usaremos o seguinte exemplo:

In [9]:
sentence = 'JoHN lIkEs to WaTch MOviEs. MaRy LiKes mOvIeS ToO. LoOK, MARy aLso LikEs tO wAtCH fOoTbaLl gAmEs!'

### Caixa Baixa
Mesmo sendo uma etapa simples, é importante para que o classificador não entenda duas grafias de uma mesma palavra como sendo duas palavras distintas (por exemplo, `caixa` e `Caixa`).

A própria biblioteca String tem um método para fazer isso facilmente:

In [10]:
import string

lowercase = sentence.lower()

print(lowercase)

john likes to watch movies. mary likes movies too. look, mary also likes to watch football games!


### Remoção de Pontuações
As pontuações não serão úteis para o aprendizado dos classificadores usados no trabalho, então todas são retiradas.

Especialmente para um dos _datasets_ que utilizei ([Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment/)), foi necessário incrementar esta etapa, removendo também _tags_ HTML que estavam presentes nos textos.

Com o auxílio dos métodos `translate()` e `maketrans()`, substituí todas as pontuações por espaços em branco.

In [11]:
tags_removed = lowercase.replace('<br />', ' ')
punctuation_removed = tags_removed.translate(tags_removed.maketrans(string.punctuation, ' ' * len(string.punctuation)))
                                             
print(punctuation_removed)

john likes to watch movies  mary likes movies too  look  mary also likes to watch football games 


### Remoção de _Stop Words_
_Stop words_ são palavras que, sozinhas, não possuem significado. Elas sempre precisam estar acompanhadas de outras palavras para fazerem sentido. Alguns exemplos são artigos e preposições.

A biblioteca NLTK traz uma lista de _stop words_ para diversos idiomas (incluindo Português), que usamos para fazer a remoção delas de nossos conjuntos de dados, após separar cada texto em tokens. No meu trabalho, utilizei o conjunto de _stop words_ em Inglês.

In [12]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# nltk.download('punkt')
# nltk.download('stopwords')

tokens = word_tokenize(punctuation_removed)
list_stopwords = set(stopwords.words('english'))
stopwords_removed = [word for word in tokens if word not in list_stopwords]

print(stopwords_removed)

['john', 'likes', 'watch', 'movies', 'mary', 'likes', 'movies', 'look', 'mary', 'also', 'likes', 'watch', 'football', 'games']


Até este momento, os textos ficarão em forma de lista de tokens, para que sejam processadas posteriormente com a técnica escolhida. A partir daqui, os quatro _pipelines_ se dividem.

---
## Redução das Palavras
Com o tratamento dos textos feito até aqui, a próxima etapa envolve a redução da inflexão de todas as palavras do texto. Existem duas técnicas conhecidas, o _stemming_ e a lematização, que são aplicadas separadamente.

### _Stemming_
O processo de _stemming_, em resumo, retira o sufixo da palavra e reduz ela ao seu radical.

Há vários algoritmos conhecidos para esta etapa, sendo um deles o algoritmo de Porter, que usei no meu trabalho. Ele já vem implementado na biblioteca NLTK.

In [13]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
stemmed_words = [stemmer.stem(word) for word in stopwords_removed]
stemmed_sentence = " ".join(stemmed_words)

print(stemmed_sentence)

john like watch movi mari like movi look mari also like watch footbal game


### Lematização
Já a lematização reduz as palavras à sua forma mais básica. Por exemplo, "vendido" é reduzido ao infinitivo, "vender".

A implementação aqui é um pouco mais complexa, pois para fazer a lematização, é necessária uma etapa a mais, de _Part-of-Speech Tagging_ (ou _POS Tagging_). Conforme descrevi na minha monografia (Seção 2.2.1.5), cada palavra recebe uma _tag_ morfossintática, que indica sua classificação dentro do contexto da frase. Somente após a atribuição dessas _tags_, podemos começar a lematização.

Um exemplo de _POS Tagging_ pode ser visto na figura abaixo.

<center><img src="https://d33wubrfki0l68.cloudfront.net/d5cbc4b0e14c20f877366b69b9171649afe11fda/d96a8/assets/images/bigram-hmm/pos-title.jpg" width="512"></center>

A biblioteca NLTK também contém um lematizador, mas que funciona com WordNet, um banco de dados lexical que também opera com suas próprias _tags_. De modo que possamos usar este lematizador, é preciso mapear as _tags_ do POS para as do WordNet.

Para isso, definimos o método a seguir:

In [14]:
def pos_wordnet(pos_tags):
    if pos_tags.startswith('J'):
        return wordnet.ADJ
    elif pos_tags.startswith('V'):
        return wordnet.VERB
    elif pos_tags.startswith('N'):
        return wordnet.NOUN
    elif pos_tags.startswith('R'):
        return wordnet.ADV
    else:
        return None

Então, realizamos a etapa de _POS Tagging_.

In [22]:
from nltk.tag import pos_tag

pos_tagging = pos_tag(stopwords_removed)

Com as _tags_ atribuídas, fazemos o mapeamento para as _tags_ do WordNet, usando o método `pos_wordnet()`.

In [23]:
lemmatization_tags = map(lambda x: (x[0], pos_wordnet(x[1])), pos_tagging)

Por fim, pode-se realizar a lematização em si.

In [24]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
lemmatized_words = []

for word, tag in lemmatization_tags:
    if tag is None:
        lemmatized_words.append(word)
    else:
        lemmatized_words.append(lemmatizer.lemmatize(word, tag))

lemmatized_sentence = " ".join(lemmatized_words)

print(lemmatized_sentence)

john like watch movie mary like movie look mary also like watch football game


---
## Representação dos Textos
Por fim, para que o algoritmo de _Machine Learning_ consiga interpretar, extrair padrões do nosso conjunto de dados e gerar um modelo dele, é preciso que os textos sejam representados numericamente.

Dessa forma, utilizei _Bag-of-Words_ (BoW) e TF-IDF para fazer essa conversão.

Para melhor exemplificação, usaremos 3 exemplos para demonstrar os dois métodos:

In [ ]:
data = [
    'John likes to watch movies. Mary likes movies too.',
    'Mary also likes to watch football games.',
    'John likes to watch movies. Mary likes movies too. Mary also likes to watch football games.'
]

É importante ressaltar que não aplicarei as etapas de processamento que mostrei até aqui, dessa vez. O intuito de usar estes três exemplos acima é puramente gerar uma melhor visualização da ideia das técnicas de representação.

### _Bag-of-Words_
No BoW, para cada texto, gera-se um vetor cujo tamanho equivale à quantidade de palavras contidas em todos os textos do _dataset_. Cada posição deste vetor representa uma palavra, e seu valor é dado pelo número de vezes que esse termo aparece no texto.

Para esse mapeamento do texto para os vetores, usei a biblioteca Scikit-learn.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_bow = CountVectorizer()
bag_of_words = vectorizer_bow.fit_transform(data).toarray()

A lista completa de palavras (ou atributos) fica da seguinte forma:

In [ ]:
print(vectorizer_bow.get_feature_names())

E então, o vetor gerado por _Bag-of-Words_ fica desta forma:

In [ ]:
print(bag_of_words)

### TF-IDF
Já o TF-IDF, por sua vez, calcula uma frequência relativa para cada palavra, por meio de uma proporção inversa entre sua frequência absoluta em um dado texto e a porcentagem de documentos em que este termo aparece (ver Seção 2.2.3.2 da monografia).

Esta representação também é implementada pelo Scikit-learn.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_tf_idf = TfidfVectorizer()
tf_idf = vectorizer_tf_idf.fit_transform(data).toarray()

print(tf_idf)

Note que usamos a frequência absoluta de cada palavra para calcular o TF-IDF, exatamente o que é utilizado para gerar o _Bag-of-Words_. Para situações específicas, pode ser interessante gerar o BoW primeiro, para depois transformar o vetor para a representação por TF-IDF. Isto pode ser feito com a classe `TfidfTransformer`.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

transformer_tf_idf = TfidfTransformer()
transformation = transformer_tf_idf.fit_transform(bag_of_words).toarray()

print(transformation)

---

Com isso, encerra-se a implementação das técnicas de processamento de texto aplicadas no meu trabalho. Estes vetores demonstrados acima são, então, usados nos algoritmos de classificação, para treino dos respectivos modelos.

Para fins de modularização, reuni todas estas técnicas em uma única classe Python, importando-a em meus Scripts dos modelos, para os experimentos do meu TCC. Esta classe pode ser encontrada [aqui](https://github.com/anajbellini/SentimentAnalysis-MovieReviews/blob/master/models/text_processing.py).

Os códigos do meu trabalho (principalmente dos métodos demonstrados neste Notebook) ainda podem ser bastante melhorados. Por exemplo, um meio que estudei após a apresentação do meu trabalho, para organizar toda a sequência de aplicação das técnicas de processamento, envolve usar a classe `sklearn.pipeline.Pipeline`. Futuramente, farei uma nova versão dessas implementações do meu TCC, com essa e outras melhorias.

In [25]:
---

Com isso, encerra-se a implementação das técnicas de processamento de texto aplicadas no meu trabalho. Estes vetores demonstrados acima são, então, usados nos algoritmos de classificação, para treino dos respectivos modelos.

Para fins de modularização, reuni todas estas técnicas em uma única classe Python, importando-a em meus Scripts dos modelos, para os experimentos do meu TCC. Esta classe pode ser encontrada [aqui](https://github.com/anajbellini/SentimentAnalysis-MovieReviews/blob/master/models/text_processing.py).

Os códigos do meu trabalho (principalmente dos métodos demonstrados neste Notebook) ainda podem ser bastante melhorados. Por exemplo, um meio que estudei após a apresentação do meu trabalho, para organizar toda a sequência de aplicação das técnicas de processamento, envolve usar a classe `sklearn.pipeline.Pipeline`. Futuramente, farei uma nova versão dessas implementações do meu TCC, com essa e outras melhorias.

É importante ressaltar que não aplicarei as etapas de processamento que mostrei até aqui, dessa vez. O intuito de usar estes três exemplos acima é puramente gerar uma melhor visualização da ideia das técnicas de representação.

### _Bag-of-Words_
No BoW, para cada texto, gera-se um vetor cujo tamanho equivale à quantidade de palavras contidas em todos os textos do _dataset_. Cada posição deste vetor representa uma palavra, e seu valor é dado pelo número de vezes que esse termo aparece no texto.

Para esse mapeamento do texto para os vetores, usei a biblioteca Scikit-learn.

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_bow = CountVectorizer()
bag_of_words = vectorizer_bow.fit_transform(data).toarray()

A lista completa de palavras (ou atributos) fica da seguinte forma:

In [33]:
print(vectorizer_bow.get_feature_names())

['also', 'football', 'games', 'john', 'likes', 'mary', 'movies', 'to', 'too', 'watch']


E então, o vetor gerado por _Bag-of-Words_ fica desta forma:

In [34]:
print(bag_of_words)

[[0 0 0 1 2 1 2 1 1 1]
 [1 1 1 0 1 1 0 1 0 1]
 [1 1 1 1 3 2 2 2 1 2]]


### TF-IDF
Já o TF-IDF, por sua vez, calcula uma frequência relativa para cada palavra, por meio de uma proporção inversa entre sua frequência absoluta em um dado texto e a porcentagem de documentos em que este termo aparece (ver Seção 2.2.3.2 da monografia).

Esta representação também é implementada pelo Scikit-learn.

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_tf_idf = TfidfVectorizer()
tf_idf = vectorizer_tf_idf.fit_transform(data).toarray()

print(tf_idf)

[[0.         0.         0.         0.3127806  0.48580407 0.24290204
  0.62556119 0.24290204 0.3127806  0.24290204]
 [0.42983971 0.42983971 0.42983971 0.         0.33380888 0.33380888
  0.         0.33380888 0.         0.33380888]
 [0.21484319 0.21484319 0.21484319 0.21484319 0.5005347  0.3336898
  0.42968638 0.3336898  0.21484319 0.3336898 ]]


Note que usamos a frequência absoluta de cada palavra para calcular o TF-IDF, exatamente o que é utilizado para gerar o _Bag-of-Words_. Para situações específicas, pode ser interessante gerar o BoW primeiro, para depois transformar o vetor para a representação por TF-IDF. Isto pode ser feito com a classe `TfidfTransformer`.

In [30]:
from sklearn.feature_extraction.text import TfidfTransformer

transformer_tf_idf = TfidfTransformer()
transformation = transformer_tf_idf.fit_transform(bag_of_words).toarray()

print(transformation)

[[0.         0.         0.         0.3127806  0.48580407 0.24290204
  0.62556119 0.24290204 0.3127806  0.24290204]
 [0.42983971 0.42983971 0.42983971 0.         0.33380888 0.33380888
  0.         0.33380888 0.         0.33380888]
 [0.21484319 0.21484319 0.21484319 0.21484319 0.5005347  0.3336898
  0.42968638 0.3336898  0.21484319 0.3336898 ]]


---

Com isso, encerra-se a implementação das técnicas de processamento de texto aplicadas no meu trabalho. Estes vetores demonstrados acima são, então, usados nos algoritmos de classificação, para treino dos respectivos modelos.

Para fins de modularização, reuni todas estas técnicas em uma única classe Python, importando-a em meus Scripts dos modelos, para os experimentos do meu TCC. Esta classe pode ser encontrada [aqui](https://github.com/anajbellini/SentimentAnalysis-MovieReviews/blob/master/models/text_processing.py).

Os códigos do meu trabalho (principalmente dos métodos demonstrados neste Notebook) ainda podem ser bastante melhorados. Por exemplo, um meio que estudei após a apresentação do meu trabalho, para organizar toda a sequência de aplicação das técnicas de processamento, envolve usar a classe `sklearn.pipeline.Pipeline`. Futuramente, farei uma nova versão dessas implementações do meu TCC, com essa e outras melhorias.